In [1]:
import setup

setup.init_django()

In [2]:
from decouple import config

In [3]:
from analytics.models import PageView
from blog.models import BlogPost
from rag import db as rag_db, settings as rag_settings

In [4]:
from sqlalchemy import (
    create_engine,
    inspect,
)

from llama_index.core import SQLDatabase
from llama_index.core.query_engine import NLSQLTableQueryEngine
from llama_index.core.retrievers import NLSQLRetriever

In [5]:
# initialize default LlamaIndex settings
rag_settings.init()
# get pooled Neon database string from .env or env vars
vector_database_url = rag_db.get_database_url(use_pooling=True)

In [6]:
engine = create_engine(vector_database_url)

In [7]:
inspect(engine).get_table_names()

['django_migrations',
 'django_content_type',
 'auth_permission',
 'auth_group',
 'auth_group_permissions',
 'auth_user',
 'auth_user_groups',
 'auth_user_user_permissions',
 'django_admin_log',
 'django_session',
 'blog_blogpost',
 'products_embedding',
 'products_product',
 'analytics_pageview']

In [8]:
tables = []
models = [BlogPost, PageView]
for model in models:
    table = model._meta.db_table
    tables.append(table)

In [9]:
tables

['blog_blogpost', 'analytics_pageview']

In [10]:
sql_database = SQLDatabase(engine, include_tables=tables)

/Users/cfe/Dev/talk-to-django/venv/lib/python3.11/site-packages/llama_index/core/utilities/sql_wrapper.py:110: SAWarning: Did not recognize type 'vector' of column 'embedding'
  self._metadata.reflect(


In [11]:
sql_query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=tables,
)

In [12]:
response = sql_query_engine.query("What blog post has the most views?")
print(str(response))

/Users/cfe/Dev/talk-to-django/venv/lib/python3.11/site-packages/llama_index/core/utilities/sql_wrapper.py:167: SAWarning: Did not recognize type 'vector' of column 'embedding'
  for column in self._inspector.get_columns(table_name, schema=self._schema):


The blog post with the most views is titled "Taking it very seriously," which has garnered a total of 2,484 views.


In [15]:
for node in response.source_nodes:
    print(node.node.get_content())

[('Blog Post 1', 1523)]


In [18]:
nl_sql_retriever = NLSQLRetriever(
    sql_database, tables=tables, return_raw=True
)

r = nl_sql_retriever.retrieve("What is my least most viewed blog post?")

In [19]:
print(r)
for node in r:
    print(node)
    print(node.metadata)

[NodeWithScore(node=TextNode(id_='df4d692b-8c86-4094-a258-4784d067b810', embedding=None, metadata={'sql_query': 'SELECT blog_blogpost.id, blog_blogpost.title, COUNT(analytics_pageview.id) AS view_count\nFROM blog_blogpost\nLEFT JOIN analytics_pageview ON blog_blogpost.id = analytics_pageview.post_id\nGROUP BY blog_blogpost.id\nORDER BY view_count ASC\nLIMIT 1;', 'result': [(29, 'New Blog Post', 0)], 'col_keys': ['id', 'title', 'view_count']}, excluded_embed_metadata_keys=['sql_query', 'result', 'col_keys'], excluded_llm_metadata_keys=['sql_query', 'result', 'col_keys'], relationships={}, text="[(29, 'New Blog Post', 0)]", mimetype='text/plain', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=None)]
Node ID: df4d692b-8c86-4094-a258-4784d067b810
Text: [(29, 'New Blog Post', 0)]
Score: None

{'sql_query': 'SELECT blog_blogpost.id, blog_blogpost.title, COUNT(analytics_pageview.id) AS vi